# RSA Stream

> CRYPTO | 100 | 121 Solves
>
> I made a stream cipher out of RSA! But people say I made a huge mistake. Can you decrypt my cipher?
> 
> Attachment: rsa_stream.tar.gz (md5 0b54cd0e8cff0ee8507e5bc9c7cc503e)

- XOR `chal.enc` with `chal.py` to recover cipher stream
- From cipher stream we have:
$$
c_1 = m^{65537} \mod n \\
c_2 = m^{65539} \mod n \\
c_3 = m^{65543} \mod n
$$
- Rearrange to get m
$$
\begin{align*}
c_2 c_1^{-1} & = m^{65539} m^{-65537} \mod n \\
& = m^2 \mod n \\
c_1 (c_2 c_1^{-1})^{-32768} & = m^{65537} (m^2)^{-32768} \mod n \\
& = m \mod n
\end{align*}
$$
- Convert to flag

Flag: `ACSC{changing_e_is_too_bad_idea_1119332842ed9c60c9917165c57dbd7072b016d5b683b67aba6a648456db189c}`

# Filtered

> PWN | 100 | 168 Solves
> 
> Filter invalid sizes to make it secure!
> 
> `nc filtered.chal.acsc.asia 9001`
> Backup: `nc 167.99.78.201 9001`
>
> Attachment: filtered.tar.gz (md5 9a6cb1b3eafce70ff549ba6b942f34a9)

- Send `-1` as length to read, type casting to unsigned int will allow us to read enough data to cause buffer overflow
- Smash stack and change return address

Flag: `ACSC{GCC_d1dn'7_sh0w_w4rn1ng_f0r_1mpl1c17_7yp3_c0nv3rs10n}`

# NYONG Coin

> FORENSICS | 140 | 26 Solves
> 
> 'Kim' is the CEO of a cryptocurrency exchange 'A'.  
> He asked the management team for the full transaction history of 'NYONG coin' traded in a specific period.  
> And here is 'Lee', a member of the management team who hates 'Kim', delivered a maliciously manipulated transaction history to 'Kim' in a USB.  
> Analyze the USB and find one manipulated transaction in there!  
> Flag Format: ACSC{lowercase(MANIPULATED_TRANSACTION_ID)}
> 
> Download Link 1: https://1drv.ms/u/s!At0nZXK3fObIgoQAMtilBAZd017Klg?e=7VKBqz  
> Download Link 2: https://drive.google.com/file/d/10SfzE59dD88A_TFZTxfAvqoBwsO18dxU  
> SHA-1: 2E21641DCE2A37959D1010E1637B8FFAEC8CF627

- Open `.E01` file in autopsy
- Carve 10 `.xlsx` documents in filesystem
- Find suspicious strings inside unallocated space and carve 1 `.xlsx` document, this is the 'clean' version of the modified doucment
- Find which (modified) doucment from filesystem corresponds to the clean document
- Import both with `pd.read_excel()`, then diff the contents
- Find two entries with different amount and transaction ID

Clean:

```
40990,2020-07-28 16:55:58,8d77a554-dc64-478c-b093-da4493a8534c,NYONG,***** USD,7151.7 NYONG,E,rlciooedxtiyotrtnzbsbdtbezsstrilfqbflbgoupvxpfzaicrwupuzfqilsrph
```

Modified:
```
40990,2020-07-28 16:55:58,8d77a554-dc64-478c-b093-da4493a8534d,NYONG,***** USD,6151.7 NYONG,E,rlciooedxtiyotrtnzbsbdtbezsstrilfqbflbgoupvxpfzaicrwupuzfqilsrph
```

Flag: `ACSC{8d77a554-dc64-478c-b093-da4493a8534d}`

# CBCBC

> CRYPTO | 210 | 35 Solves
> 
> Wow, free flags! But encrypted with CBC... twice?
> 
> `nc cbcbc.chal.acsc.asia 52171`
> Backup: `nc 167.99.77.49 52171`
> 
> Attachment: cbcbc.tar.gz (md5 2e097805ffb62dff8c1f1ab5040c863b)

- Different error message is thrown when padding fails or json parse fails (or possible successful login), giving a padding oracle
- Solve 3 blocks at a time due to double AES scheme

Flag: `ACSC{wow_double_CBC_mode_cannot_stop_you_from_doing_padding_oracle_attack_nice_job}`

# Pickle Rick

> REV | 220 | 23 Solves
> 
> I found a suspicious pickle file with a script...
> Wait, who is Pickle Rick?
> 
> Attachment: pickle_rick.tar.gz (md5 0d75a7301b9341864d0efbda4c644f14)

- We have `.py` file that opens a `.pickle`, which then runs flag-checking code
- Make a local copy of the [pickle module](https://github.com/python/cpython/blob/3.9/Lib/pickle.py) and import the `.pickle` with this hooked module
  - Remember to disable the fallback (?native) unpickler around line 1777
- Make local changes which logs when various functions are called, especially the REDUCE (`R`) opcode which is run whenever functions are called

```python
def load_reduce(self):
    stack = self.stack
    args = stack.pop()
    func = stack[-1]
    if func != print:
        stack[-1] = func(*args)
    a = str(args)
    if len(a) > 30:
        a = a[:25] + '...'
    b = str(stack[-1]) if func != print else None
    if b is not None and len(b) > 30:
        b = b[:25] + '...'
    print(func, f"{a} => {b}")
dispatch[REDUCE[0]] = load_reduce
```

- The code sets up two new functions called `mix` and `search` by using the `code` constructor

```
<built-in function print> ('\n                     ... => None
<built-in function print> ('Pickle Rick says:',) => None
<built-in function print> (b'Wubba lubba dub-dub!!',) => None
<built-in function print> ('The flag machine says:',) => None
<class 'type'> (<function amazing_functi... => <class 'function'>
<built-in function getattr> (<function amazing_functi... => <code object amazing_func...
<class 'type'> (<code object amazing_fun... => <class 'code'>
<class 'code'> (2, 0, 0, 5, 6, 67, b'd\x... => <code object search at 0x...
<class 'function'> (<code object search at 0... => <function search at 0x7f3...
<class 'type'> (<function amazing_functi... => <class 'function'>
<built-in function getattr> (<function amazing_functi... => <code object amazing_func...
<class 'type'> (<code object amazing_fun... => <class 'code'>
<class 'code'> (1, 0, 0, 6, 5, 67, b'|\x... => <code object mix at 0x7f3...
<class 'function'> (<code object mix at 0x7f... => <function mix at 0x7f3611...
```

- `mix` is called once, then `search` for each character in flag

```
<function mix at 0x7f361115a040> (b'Wubba lubba dub-dub!!',) => [77, 84, 207, 188, 169, 1...
<function amazing_function at 0x7f361137a0d0> ([77, 84, 207, 188, 169, ... => 77
<function search at 0x7f3611154f70> ((((((((((115,), (99,)), ... => 186
<function amazing_function at 0x7f361137a0d0> ([77, 84, 207, 188, 169, ... => 84
<function search at 0x7f3611154f70> ((((((((((115,), (99,)), ... => 184
[... more calls to search and amazing_function]
<function search at 0x7f3611154f70> ((((((((((115,), (99,)), ... => 27
<function amazing_function at 0x7f361137a0d0> ([77, 84, 207, 188, 169, ... => 176
<function search at 0x7f3611154f70> ((((((((((115,), (99,)), ... => 86
<function amazing_function at 0x7f361137a0d0> ((186, 184, 21, 250, 11, ... => WRONG!
<built-in function print> ('WRONG!',) => None
```

- By hooking into the `code` constructor we can leak the python bytecode, then dissassemble it and try to recreate the functions

```python
def search(a, b):
  c = 0
  while True:
    try:
      a0, a1 = a
      if b % 2 == c:
        a = a1
        b = b // 2
        c = 1 - c
      else:
        a = a0
        b = b // 2
        c = 1 - c
    except:
      return a[0]

def mix(a):
  ln = len(a)
  arr = []
  i = 0
  while i < ln:
    s, j = 0, 0
    while j < ln:
      s += (j + 1) * a[(i + j) % ln]
      j += 1
    s = s % 257
    assert(s < 256)
    arr.append(s)
    i += 1
  return arr
```

- By differential analysis we can also see that the "binary tree" used in `search` and the final sequence that is checked to see if flag is correct does not change with input

```python
search_data = (((((((((115,),...
goal = [53, 158, 33, 115, ...
```

- `search` is a one to one mapping, and we can recover the sbox used by trying all values of `b`
- `mix` is effectively a matrix multiplication
- Recover the original flag with sagemath `matrix.inverse()`

```python
unsearch = {}
for i in range(256):
  unsearch[search(search_data, i)] = i
postmix = [unsearch[i] for i in goal]

F = GF(257)
d = vector(F, list(b"Xbcdefghijklmnopqrstu"))
m = [[0 for _ in range(21)] for _ in range(21)]

for i in range(21):
  for j in range(21):
    m[(i + j) % 21][i] = j + 1
m = matrix(F, m)
print(''.join([chr(i) for i in (vector(F, postmix) * m.inverse()).list()]))
```

Flag: `ACSC{YEAH!I'm_pickle-RICK!}` _lol nice_